## Built-in Tool - DuckDuckGo Search

In [7]:
from langchain_community.tools import DuckDuckGoSearchResults

tool = DuckDuckGoSearchResults()
tool.invoke("Obama")


UnboundLocalError: cannot access local variable 'results' where it is not associated with a value

In [ ]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [8]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
'ls' is not recognized as an internal or external command,
operable program or batch file.



c:\Users\Sai12\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

In [9]:
from langchain_core.tools import tool

In [ ]:
# Step 1 - create a function
## add docs strings so that the tool can be used in LLMs
def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [11]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [ ]:
# Step 3 - add tool decorator
##adding a decortor makes the function special so that it intreacts with LLM's
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [13]:
result = multiply.invoke({"a":3, "b":5})

In [14]:
print(result)

15


In [16]:
multiply

StructuredTool(name='multiply', description='Multiply two numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002DE4F08EFC0>)

In [17]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [18]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [19]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [22]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number")
    b: int = Field(required=True, description="The second number")

In [23]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [24]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func, ## calling the function
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput ## passing the Pydantic class
)

In [25]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [26]:
from langchain.tools import BaseTool
from typing import Type

In [27]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [32]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput ## passing the Pydantic class

    def _run(self, a: int, b: int) -> int: ## this is an abstact method which needs to be implemented as we are inheriting from BaseTool
        return a * b

In [33]:
multiply_tool = MultiplyTool()

In [34]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [35]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [36]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [37]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers


In [38]:
tools[0].invoke({"a":5, "b":10})  # Using the first tool (add)

15

In [ ]:
tools[1].invoke({"a":5, "b":10})  # Using the first tool (multiply)

50